# Data Processing and Feature Engineering

This notebook performs data preprocessing and feature engineering on the German Credit Dataset. We'll transform raw features into model-ready inputs while preserving business meaning.

## Objectives:
- Load and validate raw data
- Engineer new features from existing variables
- Transform temporal features to numeric formats
- Prepare categorical and numerical feature sets
- Save processed data for modeling

---

## 1. Setup and Configuration

In [16]:
# Core data science libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import re
from scipy.stats import chi2_contingency
from typing import Optional, List, Union, Tuple

# Project utilities
from test_case_fairmoney.utils.constants import PATH_DATA_RAW, PATH_DATA_COMPRESSED

# Configuration
pd.set_option("display.max_rows", 300)
pd.set_option("display.max_columns", 800)
pd.set_option("display.float_format", "{:.2f}".format)
warnings.filterwarnings("ignore")

# Plotting style
plt.style.use("default")
sns.set_palette("husl")

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


## 2. Data Loading and Initial Assessment

In [17]:
# Load the German Credit dataset
print("📊 Loading German Credit Dataset...")
df = pd.read_csv(PATH_DATA_RAW / "credit.csv").drop(columns=["Unnamed: 0"])

print(f"✅ Dataset loaded successfully!")
print(f"   Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

📊 Loading German Credit Dataset...
✅ Dataset loaded successfully!
   Shape: 1,000 rows × 22 columns


## 3. Feature Engineering

### 3.1 Temporal Features: Convert Duration Strings to Months

Many features contain duration information in string format (e.g., "2 years", "8 months"). We'll convert these to numeric months for modeling.

In [18]:
def convert_residence_to_months(residence_str):
    """
    Convert residence/employment history string to numeric months.

    Examples:
        "2 years" -> 24
        "8 months" -> 8
        "1 years 6 months" -> 18
        "0 months" -> 0

    Args:
        residence_str: String containing duration information

    Returns:
        int: Total duration in months
    """
    if pd.isna(residence_str):
        return np.NaN

    residence_str = str(residence_str).lower().strip()
    total_months = 0

    # Extract years and convert to months
    years_match = re.search(r"(\d+)\s*years?", residence_str)
    if years_match:
        years = int(years_match.group(1))
        total_months += years * 12

    # Extract months
    months_match = re.search(r"(\d+)\s*months?", residence_str)
    if months_match:
        months = int(months_match.group(1))
        total_months += months

    return total_months


print("🔧 Engineering temporal features...")

# Apply conversion to residence and employment history
df["residence_months"] = (
    df["residence_history"].apply(convert_residence_to_months).replace("nan", np.NaN)
)
df["employment_months"] = (
    df["employment_length"].apply(convert_residence_to_months).replace("nan", np.NaN)
)

print("✅ Temporal features created:")
print(
    f"   • residence_months: {df['residence_months'].min():.0f} - {df['residence_months'].max():.0f} months"
)
print(
    f"   • employment_months: {df['employment_months'].min():.0f} - {df['employment_months'].max():.0f} months"
)

# Show sample conversions
print("\n📊 Sample Conversions:")
sample_df = pd.DataFrame(
    {
        "Original (residence)": df["residence_history"].head(5),
        "Converted (months)": df["residence_months"].head(5),
    }
)
display(sample_df)

🔧 Engineering temporal features...
✅ Temporal features created:
   • residence_months: 0 - 288 months
   • employment_months: 0 - 228 months

📊 Sample Conversions:


Original (residence)  Converted (months)
0              6 years               72.00
1             5 months                5.00
2              4 years               48.00
3             13 years              156.00
4             13 years              156.00

### 3.2 Telephone Feature Decomposition

Extract country code and mobile network information from telephone numbers.

In [19]:
print("📱 Engineering telephone features...")

# Extract country code (first 3 digits) and mobile network (next 3 digits)
df["telephone_country_code"] = df["telephone"].apply(
    lambda x: np.nan if pd.isna(x) else str(x)[:3]
)

df["telephone_mobile_network"] = df["telephone"].apply(
    lambda x: np.nan if pd.isna(x) else str(x)[3:6]
)
df["is_phone_nigeria"] = df["telephone_country_code"].apply(
    lambda x: 1 if x == "234" else 0
)
print("✅ Telephone features created:")
print(
    f"   • telephone_country_code: {df['telephone_country_code'].nunique()} unique codes"
)
print(
    f"   • telephone_mobile_network: {df['telephone_mobile_network'].nunique()} unique networks"
)

# Show distribution
print("\n📊 Country Code Distribution:")
print(df["telephone_country_code"].value_counts().head())

print("\n📊 Mobile Network Distribution:")
print(df["telephone_mobile_network"].value_counts().head())

📱 Engineering telephone features...
✅ Telephone features created:
   • telephone_country_code: 1 unique codes
   • telephone_mobile_network: 337 unique networks

📊 Country Code Distribution:
telephone_country_code
234    404
Name: count, dtype: int64

📊 Mobile Network Distribution:
telephone_mobile_network
121    3
417    3
707    3
954    3
077    3
Name: count, dtype: int64


### 3.3 is_checking_negative

In [20]:
df["is_checking_negative"] = (df["checking_balance"] < 0).astype(int)
print(
    f"✅ Created: is_checking_negative ({df['is_checking_negative'].mean()*100:.1f}% negative)"
)

✅ Created: is_checking_negative (27.4% negative)


## 4. Feature Set Definition

Define categorical and numerical feature sets for modeling. This separation is important for:
- Proper preprocessing strategies
- Model-specific requirements (e.g., CatBoost handles categoricals natively)
- Feature analysis and interpretation

In [13]:
print("🎯 Defining feature sets...")
print("=" * 60)

# Target variable
target = ["default"]

# Categorical features
feat_cat = [
    "credit_history",
    "purpose",
    "personal_status",
    "other_debtors",
    "property",
    "installment_plan",
    "housing",
    "foreign_worker",
    "job",
    "gender",
    "telephone_country_code",
    "telephone_mobile_network",
]

# Numerical features
feat_num = [
    "checking_balance",
    "months_loan_duration",
    "amount",
    "savings_balance",
    "installment_rate",
    "existing_credits",
    "dependents",
    "residence_months",
    "employment_months",
    "is_phone_nigeria",
    "is_checking_negative",
]

print(f"📊 Feature Set Summary:")
print(f"   Target variable: {target[0]}")
print(f"   Categorical features: {len(feat_cat)}")
print(f"   Numerical features: {len(feat_num)}")
print(f"   Total features: {len(feat_cat) + len(feat_num)}")

print(f"\n📝 Categorical Features ({len(feat_cat)}):")
for i, feat in enumerate(feat_cat, 1):
    nunique = df[feat].nunique()
    print(f"   {i:2d}. {feat:<30} ({nunique:>3} unique values)")

print(f"\n🔢 Numerical Features ({len(feat_num)}):")
for i, feat in enumerate(feat_num, 1):
    min_val = df[feat].min()
    max_val = df[feat].max()
    print(f"   {i:2d}. {feat:<30} (range: {min_val:.0f} - {max_val:.0f})")

🎯 Defining feature sets...
📊 Feature Set Summary:
   Target variable: default
   Categorical features: 12
   Numerical features: 11
   Total features: 23

📝 Categorical Features (12):
    1. credit_history                 (  5 unique values)
    2. purpose                        ( 10 unique values)
    3. personal_status                (  3 unique values)
    4. other_debtors                  (  3 unique values)
    5. property                       (  4 unique values)
    6. installment_plan               (  3 unique values)
    7. housing                        (  3 unique values)
    8. foreign_worker                 (  2 unique values)
    9. job                            (  4 unique values)
   10. gender                         (  2 unique values)
   11. telephone_country_code         (  1 unique values)
   12. telephone_mobile_network       (337 unique values)

🔢 Numerical Features (11):
    1. checking_balance               (range: -50 - 999)
    2. months_loan_duration        

## 5. Data Quality Check

Verify the processed dataset for any issues before saving.

In [14]:
print("🔍 Data Quality Assessment:")
print("=" * 60)

# Check for missing values in feature sets
all_features = feat_cat + feat_num + target
missing_summary = df[all_features].isnull().sum()
total_missing = missing_summary.sum()

if total_missing > 0:
    print("⚠️  Missing values detected:")
    print(missing_summary[missing_summary > 0])
else:
    print("✅ No missing values in selected features")

# Check for infinite values in numerical features
inf_count = np.isinf(df[feat_num]).sum().sum()
if inf_count > 0:
    print(f"⚠️  {inf_count} infinite values detected in numerical features")
else:
    print("✅ No infinite values in numerical features")

# Check for duplicate rows
duplicates = df.duplicated().sum()
if duplicates > 0:
    print(f"⚠️  {duplicates} duplicate rows detected")
else:
    print("✅ No duplicate rows")

# Summary statistics
print(f"\n📊 Final Dataset Statistics:")
print(f"   Total records: {len(df):,}")
print(f"   Total features (selected): {len(all_features)}")
print(
    f"   Memory usage: {df[all_features].memory_usage(deep=True).sum() / 1024**2:.2f} MB"
)

🔍 Data Quality Assessment:
⚠️  Missing values detected:
personal_status             310
telephone_country_code      596
telephone_mobile_network    596
checking_balance            394
savings_balance             183
residence_months            130
employment_months            62
dtype: int64
✅ No infinite values in numerical features
✅ No duplicate rows

📊 Final Dataset Statistics:
   Total records: 1,000
   Total features (selected): 24
   Memory usage: 0.78 MB


## 6. Save Processed Dataset

Save the processed dataset with engineered features for use in modeling.

In [15]:
print("💾 Saving processed dataset...")

# Select only relevant features
df_processed = df[feat_cat + feat_num + target].copy()

# Save to parquet format (efficient compression)
output_path = PATH_DATA_COMPRESSED / "credit_processed.parquet"
df_processed.to_parquet(output_path, index=False)

# Verify save
df_verify = pd.read_parquet(output_path)
print(f"✅ Dataset saved successfully!")
print(f"   📁 Location: {output_path}")
print(f"   📊 Shape: {df_verify.shape[0]:,} rows × {df_verify.shape[1]} columns")
print(f"   💾 File size: {output_path.stat().st_size / 1024:.2f} KB")
print(f"   🔍 Verification: {len(df_verify)} records loaded")

💾 Saving processed dataset...
✅ Dataset saved successfully!
   📁 Location: /Users/phamkythoai/Desktop/Data_Science/test_case_fairmoney/project_data/data/compressed/credit_processed.parquet
   📊 Shape: 1,000 rows × 24 columns
   💾 File size: 34.60 KB
   🔍 Verification: 1000 records loaded
